In [1]:
import time

import glow
spark = glow.register(spark)
import pyspark.sql.functions as fx
from pyspark.sql.types import *
from pyspark.sql.functions import *
from random import sample

In [2]:
vcf_path_glob = 'data/*0R.strelka.somatic.snvs.vcf.gz'
vcf_path = 'data/test.strelka.somatic.snvs.vcf.gz'
#phenotypes_path = '/databricks-datasets/genomics/1000G/phenotypes.normalized'
#reference_genome_path = "/dbfs/databricks-datasets/genomics/grch37/data/human_g1k_v37.fa"
#vcf_output_path = "dbfs:/home/genomics/vcf/subset.vcf"


In [3]:
vcf = (
  spark
  .read
  .format("vcf")
  .load(vcf_path) # , includeSampleIds=True
)

vcf.printSchema()

WARNING	2021-07-15 10:28:40	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream


root
 |-- contigName: string (nullable = true)
 |-- start: long (nullable = true)
 |-- end: long (nullable = true)
 |-- names: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- referenceAllele: string (nullable = true)
 |-- alternateAlleles: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- qual: double (nullable = true)
 |-- filters: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- splitFromMultiAllelic: boolean (nullable = true)
 |-- INFO_SomaticEVS: double (nullable = true)
 |-- INFO_QSS_NT: integer (nullable = true)
 |-- INFO_PNOISE2: double (nullable = true)
 |-- INFO_MQ0: integer (nullable = true)
 |-- INFO_ReadPosRankSum: double (nullable = true)
 |-- INFO_TQSS: integer (nullable = true)
 |-- INFO_SNVSB: double (nullable = true)
 |-- INFO_DP: integer (nullable = true)
 |-- INFO_PNOISE: double (nullable = true)
 |-- INFO_QSS: integer (nullable = true)
 |-- INFO_MQ: double (nullable = true)
 |-- INFO

## Data lake

In [4]:
delta_output_path = "delta/test-delta"

(
  vcf
  .write
  .format("delta")
  .mode("overwrite")
  .save(delta_output_path)
)

delta_vcf = spark.read.format("delta").load(delta_output_path)

WARNING	2021-07-15 10:29:14	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2021-07-15 10:29:14	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2021-07-15 10:29:14	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream
WARNING	2021-07-15 10:29:14	AsciiLineReader	Creating an indexable source for an AsciiFeatureCodec using a stream that is neither a PositionalBufferedStream nor a BlockCompressedInputStream


In [5]:
delta_vcf.show(n=2)

+----------+--------+--------+-----+---------------+----------------+----+--------+---------------------+---------------+-----------+------------+--------+-------------------+---------+----------+-------+-----------+--------+-------+--------+------------+-------+------------+--------------------+
|contigName|   start|     end|names|referenceAllele|alternateAlleles|qual| filters|splitFromMultiAllelic|INFO_SomaticEVS|INFO_QSS_NT|INFO_PNOISE2|INFO_MQ0|INFO_ReadPosRankSum|INFO_TQSS|INFO_SNVSB|INFO_DP|INFO_PNOISE|INFO_QSS|INFO_MQ|INFO_SGT|INFO_SOMATIC|INFO_NT|INFO_TQSS_NT|           genotypes|
+----------+--------+--------+-----+---------------+----------------+----+--------+---------------------+---------------+-----------+------------+--------+-------------------+---------+----------+-------+-----------+--------+-------+--------+------------+-------+------------+--------------------+
|        14|39575774|39575775| null|              G|             [T]|null|[LowEVS]|                false| 

## Timings

In [9]:
import time

start = time.time()
vcf.filter(array_contains(col("filters"), 'PASS')).count()
print(time.time() - start)

start = time.time()
delta_vcf.filter(array_contains(col("filters"), 'PASS')).count()
print(time.time() - start)



1.8226094245910645
1.2743875980377197


In [10]:
start = time.time()
vcf.filter(array_contains(col("filters"), 'PASS')).select(element_at(col("genotypes"),1)).collect()
print(time.time() - start)

start = time.time()
delta_vcf.filter(array_contains(col("filters"), 'PASS')).select(element_at(col("genotypes"),1)).collect()
print(time.time() - start)



3.828608989715576
3.2344350814819336


In [55]:
start = time.time()
vcf.select(col("INFO_SomaticEVS"), col("referenceAllele")).collect()
print(time.time() - start)

start = time.time()
delta_vcf.select(col("INFO_SomaticEVS"), col("referenceAllele")).collect()
print(time.time() - start)


4.461416482925415
3.6080989837646484


In [49]:
start = time.time()
delta_vcf.count()
print(time.time() - start)

start = time.time()
vcf.count()
print(time.time() - start)

0.37714171409606934
1.2829225063323975


In [13]:
#genotype = delta_vcf.where((fx.col("contigName") == '22') & 
#                           (fx.col("start") == 1234567)). \
#                     selectExpr("contigName", "start", "filter(genotypes, g -> g.sampleId = '{0}') as genotypes".format(sample_id))

770269